In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Load dataset
df = pd.read_csv("synthetic_air_quality_diphtheria.csv")

# Splitting features and target variable
X = df[['SO2', 'NO2', 'PM10', 'PM2.5']]
y = df['Diphtheria']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models and hyperparameter grids
param_grid = {
    "Logistic Regression": {
        "model": LogisticRegression(),
        "params": {"C": [0.01, 0.1, 1, 10], "solver": ["lbfgs", "liblinear"]}
    },
    "Decision Tree": {
        "model": DecisionTreeClassifier(),
        "params": {"max_depth": [3, 5, 10], "min_samples_split": [2, 5, 10]}
    },
    "Random Forest": {
        "model": RandomForestClassifier(),
        "params": {"n_estimators": [50, 100, 200], "max_depth": [3, 5, 10]}
    },
    
}

# Hyperparameter tuning and training
best_model = None
best_accuracy = 0
best_params = None
best_model_name = ""

for name, config in param_grid.items():
    grid = GridSearchCV(config["model"], config["params"], cv=5, scoring='accuracy')
    grid.fit(X_train, y_train)
    
    best_model_for_name = grid.best_estimator_
    y_pred = best_model_for_name.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"{name}: {accuracy:.4f} with params {grid.best_params_}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = best_model_for_name
        best_params = grid.best_params_
        best_model_name = name

print(f"\nBest Model: {best_model_name} with accuracy {best_accuracy:.4f}")

# Save the best model
with open("best_model.pkl", "wb") as file:
    pickle.dump(best_model, file)

# Save the scaler
with open("scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

Logistic Regression: 0.7143 with params {'C': 0.01, 'solver': 'lbfgs'}
Decision Tree: 0.6538 with params {'max_depth': 3, 'min_samples_split': 10}
Random Forest: 0.6593 with params {'max_depth': 3, 'n_estimators': 50}

Best Model: Logistic Regression with accuracy 0.7143


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    RBF, ExpSineSquared, DotProduct, ConstantKernel, WhiteKernel
)
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

# 1. Load the dataset
df = pd.read_csv('diptheria_air_quality.csv')

# 2. Feature engineering: add a date column and day-of-year
df['date'] = pd.date_range(start='2020-01-01', periods=len(df), freq='D')
df['day_of_year'] = df['date'].dt.dayofyear

# 3. Define feature matrix X and target y
features = ['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3', 'day_of_year']
X = df[features].values
y = df['Diphtheria_Cases'].values

# 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 5. Clip negatives and log-transform the training target
y_train_clipped = np.clip(y_train, 0, None)
y_train_log = np.log1p(y_train_clipped)

# 6. Define composite GPR kernel with a learnable noise term
kernel = (
    ConstantKernel(1.0, (1e-3, 1e3)) * RBF(length_scale=50, length_scale_bounds=(1e-2, 1e2)) +
    ConstantKernel(1.0, (1e-3, 1e3)) * ExpSineSquared(
        length_scale=50, periodicity=365, periodicity_bounds=(1, 500)
    ) +
    ConstantKernel(1.0, (1e-3, 1e3)) * DotProduct(sigma_0=1.0, sigma_0_bounds=(1e-3, 1e3)) +
    WhiteKernel(noise_level=1.0, noise_level_bounds=(1e-5, 1e1))
)

# 7. Train the GPR model
gpr = GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=42)
gpr.fit(X_train, y_train_log)

# 8. Make predictions and back-transform
y_pred_log, y_std = gpr.predict(X_test, return_std=True)
y_pred = np.expm1(y_pred_log)
y_pred = np.clip(y_pred, 0, None)

# 9. Evaluate GPR performance
mse_gpr = mean_squared_error(y_test, y_pred)
r2_gpr = r2_score(y_test, y_pred)
print(f"GPR with noise & log-transform -> MSE: {mse_gpr:.3f}, R²: {r2_gpr:.3f}")
print("Learned kernel parameters:", gpr.kernel_)

# 10. Random Forest baseline for comparison
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_pred_rf = np.clip(y_pred_rf, 0, None)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest -> MSE: {mse_rf:.3f}, R²: {r2_rf:.3f}")


GPR with noise & log-transform -> MSE: 30.369, R²: 0.586
Learned kernel parameters: 1**2 * RBF(length_scale=50) + 1**2 * ExpSineSquared(length_scale=50, periodicity=365) + 1**2 * DotProduct(sigma_0=1) + WhiteKernel(noise_level=1)
Random Forest -> MSE: 29.943, R²: 0.592


In [3]:
import pandas as pd

# Create DataFrame of predictions vs actuals for the test set
df_pred = pd.DataFrame(X_test, columns=features)
df_pred['Actual_Cases'] = y_test
df_pred['GPR_Predicted'] = y_pred
df_pred['RF_Predicted'] = y_pred_rf

# Show the first 10 rows to the user
print("Test Set: Actual vs Predicted Diphtheria Cases")
print(df_pred.head(10))


Test Set: Actual vs Predicted Diphtheria Cases
        PM2.5        PM10        NO2        SO2        CO          O3  \
0   98.276792   96.124879  25.012908  53.180623  4.278139   58.021386   
1  123.241907  128.196473   6.698024  69.298172  8.790518   90.994288   
2   15.322354   14.823408  28.486952  59.158520  6.762370  142.662136   
3   40.098014   34.240258  76.508081  75.082710  2.474754   80.805753   
4  136.597139   32.920816  99.516243   7.415060  7.953287   10.022898   
5   54.416513  136.936176  37.015302  45.296626  8.911961   35.771163   
6    8.374431  135.195549  53.907649  41.652123  7.910953   44.795619   
7   38.050599  127.468407  67.247988  15.673630  9.842692   74.144069   
8   15.809843  181.790152  38.689105  50.661027  1.730593  146.592066   
9  117.686303  124.098203  52.231603   8.910834  2.775452  150.676074   

   day_of_year  Actual_Cases  GPR_Predicted  RF_Predicted  
0        362.0     16.676436       8.396157      8.751237  
1         74.0     22.730414 